In [2]:
import torch 
import numpy as np 
from torch.utils.data import DataLoader , TensorDataset
import torchvision.transforms as Transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import torch.nn as nn 
import torch.optim as optim

### autoGrad mechanism working under the hoos of differential equation of Jaccobian matrix 

* here all most model learing step is going from updating the wieghts gradients of function loss respect to non-linear activation funtion in input and output 
* pytorch has .Backward() that takes all the input data in vectors space and diffrential the gradients each step respect to loss of Gradiant decent and this called Autograd 

In [2]:
### let define our function here to derive 
def f(x):
    return x * 2 - x**2 -4 

def check_rate_of_change(f,x,h):
    return (f(x + h) - f(x)) / h 
            
h = 0.1 
            
for i in range(4):            
    print(f"rate of change {h:.6f} and numrical limt {check_rate_of_change(f,1,h):.6f}")
    h *= 0.1

rate of change 0.100000 and numrical limt -0.100000
rate of change 0.010000 and numrical limt -0.010000
rate of change 0.001000 and numrical limt -0.001000
rate of change 0.000100 and numrical limt -0.000100


In [6]:
### here the most use case of ato Grad is automatic differrential function respect to all the gradiants in at ones 
#Ex -
x = torch.rand((2,3),requires_grad=True)
y = x.view(-1)
print(x.shape)
ydot = torch.dot(y,y)


torch.Size([2, 3])


In [7]:
ydot.backward()
print(ydot.grad)

None


In [8]:
y.grad == y * 2

False

In [7]:
#### here we use gradiants passing with autograd 
###### tow vectors a =[2,3] ; b = [6,4]
a = torch.tensor([2.,3.],requires_grad=True)
b = torch.tensor([6.,4.],requires_grad=True)
Q = 3*a**2 + 2*b ## ==> 
print(Q)
extrenal_grad = torch.tensor([0,1])

Q.backward(gradient = extrenal_grad)

print(a.grad , b.grad)


tensor([24., 35.], grad_fn=<AddBackward0>)
tensor([ 0., 18.]) tensor([0., 2.])


#### pipline builing the model in pytorch 
- here there's pipline step to build the model for training in cerrnet task that takes input---> and provide an output 



In [8]:
def generated_data(w , b , examples):
    ### create tensors with 1d using normal distrubtion ###
    X = torch.normal(0,1,size=(examples,len(w)))
    Y = torch.matmul(X,w) + b 
    Y += torch.normal(1.0,0.01 , size=Y.shape)
    return X , Y.reshape(-1,1)

wieghts = torch.tensor([2.3,1])
bias = 0.1 
featuers_X , labels = generated_data(wieghts,bias,1000)
#targats = labels.view(-1,2)
featuers_X.shape , labels.shape #, labels.view(-1,2).shape


(torch.Size([1000, 2]), torch.Size([1000, 1]))

In [9]:
def data_loading(data, batch_size, is_train=True):
    samples = TensorDataset(*data)
    data_loading = DataLoader(samples , batch_size, shuffle=is_train)
    return data_loading

In [10]:
batch_size = 10
dataset = data_loading((featuers_X,labels),batch_size)

In [11]:
class Net(nn.Module):
    def __init__(self, input_features , output_featues):
        super(Net, self).__init__()
        self.linear = nn.Sequential(nn.Linear(input_features,output_featues,bias=True))
        #return  self.linear[0].weight , self.linear[0].bias
    def forward(self,input_):
        input_ = self.linear(input_)
        return input_
    
input_features = 20
output_featues= 2
input_ = torch.rand((5,2),requires_grad=True)
#print(input_.shape)
input_f = input_.view(-1)
print(input_f.shape)
net = Net(input_features , output_featues)
#out=net(input_f)


torch.Size([10])


In [12]:
neto = nn.Sequential(nn.Linear(2,1))
neto[0].weight , neto[0].bias

(Parameter containing:
 tensor([[-0.6155, -0.5451]], requires_grad=True),
 Parameter containing:
 tensor([0.5218], requires_grad=True))

In [13]:
Criterion = nn.MSELoss()
optimizer = optim.SGD(neto.parameters(),lr=0.01)


In [14]:
Epoch = 3 

for step in range(Epoch):
    for X,y in dataset:
        X= X.reshape(-1)
        pred = net(X)
        loss = Criterion(pred , y)
        optimizer.zero_grad()
        loss.backward()
        print(f" corrspnding to loss eval {loss.item()}")
        optimizer.step()
#     loss = Criterion(featuers_X, labels)    
#     print(f"epoch {step} loss with stpes {loss.item()}")    
        
        
    
    

 corrspnding to loss eval 5.663969993591309
 corrspnding to loss eval 9.35481071472168
 corrspnding to loss eval 7.496621131896973
 corrspnding to loss eval 17.971149444580078
 corrspnding to loss eval 10.228344917297363
 corrspnding to loss eval 7.050827980041504
 corrspnding to loss eval 13.498127937316895
 corrspnding to loss eval 4.2512407302856445
 corrspnding to loss eval 7.02276611328125
 corrspnding to loss eval 7.1733856201171875
 corrspnding to loss eval 6.404366970062256
 corrspnding to loss eval 5.77640962600708
 corrspnding to loss eval 13.203216552734375
 corrspnding to loss eval 5.975814342498779
 corrspnding to loss eval 18.6850528717041
 corrspnding to loss eval 5.645538330078125
 corrspnding to loss eval 4.136396884918213
 corrspnding to loss eval 3.64733624458313
 corrspnding to loss eval 8.52076530456543
 corrspnding to loss eval 9.385309219360352
 corrspnding to loss eval 3.183586597442627
 corrspnding to loss eval 13.018678665161133
 corrspnding to loss eval 7.138

/home/yunus/brain_tumor/Unet/lib/python3.9/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([10, 1])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [15]:
### MESloss 

def MSELoss(input_true , output_target, num_sampels):
    out =  1/num_sampels * (sum(output_target - input_true))**2
    return f"value estimes between target is {out:.4f}" 

In [16]:
m_in = torch.tensor([1,3])
m_ou = torch.tensor([2,3])
MSELoss(m_in, m_ou,2)

'value estimes between target is 0.5000'

#### CrossEntery loss with multi-labels classes 
* thid type of loss is automaticlly pass the softmax activation to estimate the probability of class to correspnding label


In [32]:
#### loading the dataset fahison 
Grayscale_channel = 3
def Dataloader(root,batches, Trans=False):
    if Trans == True:
        Trans= Transforms.Compose([
            Transforms.Resize((32,32)),
            Transforms.Grayscale(num_output_channels=Grayscale_channel),
            Transforms.ToTensor(),
            Transforms.Normalize(mean=(0.5,0.5,0.5),std=(0.5, 0.5, 0.5))
            ])
    train_set = datasets.FashionMNIST(root=root, train=True , transform=Trans , download= False)
    test_set = datasets.FashionMNIST(root=root, train=False , transform=Trans , download= False)
    traning_set = DataLoader(dataset = train_set, batch_size = batches , shuffle=True,  num_workers = 1)
    testing_set = DataLoader(dataset = test_set, batch_size = batches , shuffle=False,  num_workers = 1)
    return traning_set,testing_set

In [33]:
Data_T , Data_Ts = Dataloader("data/", 32,Trans=True) 

In [34]:
#Data_T , Data_Ts = iter(dataloader)
# for i in range(iterations):     
#     try:
#         X, Y = next(dataloader_iterator)
#     except:
#         dataloader_iterator = iter(train_loader)
#         X, Y = next(dataloader_iterator)
#     do_backprop(X, Y)
for X , y in iter(Data_Ts):
    print(f"shapa of data samples {X.shape}, the labels size is : {y.shape}")

shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples

shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples

shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples

shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([32, 3, 32, 32]), the labels size is : torch.Size([32])
shapa of data samples torch.Size([16, 3, 32, 32]), the labels size is : torch.Size([16])


In [56]:
class Net(nn.Module):
    def __init__(self, input_features , output_featues): # input_ img == [3,32,32] 2D channle 1 1024 , matxix imput
        super(Net, self).__init__()
        self.linear = nn.Linear(input_features,10,bias=True)
        self.out = nn.Linear(10,output_featues)
        #return  self.linear[0].weight , self.linear[0].bias
    def forward(self,input_):
        input_ = input_.reshape(input_.shape[0],-1)
        #input_ = input_.permute(0,1)
        print(input_.shape)
        input_ = self.linear(input_)
        print(input_.shape)
        input_ - self.out(input_)
        return input_
    
input_features =3072
input_f= torch.rand(32,3,32,32)
input_f = input_f
output_featues= 10
model = Net(input_features, output_featues)
model(input_f)

torch.Size([32, 3072])
torch.Size([32, 10])


tensor([[-4.5565e-01, -6.2048e-02,  3.1897e-01,  7.7925e-01,  9.7524e-02,
         -2.3531e-01, -3.0509e-02, -6.9276e-01, -2.6018e-01, -1.6166e-03],
        [-1.5943e-01,  1.5790e-01, -1.1346e-01,  4.8905e-01,  1.4969e-01,
          2.8849e-01,  1.4422e-01, -1.2593e+00, -2.7482e-01, -1.7318e-01],
        [-4.7633e-01, -1.8580e-01, -2.1712e-01,  2.1395e-01,  1.9826e-01,
          4.6004e-02,  9.0717e-02, -8.4497e-01, -1.0842e-01, -2.2527e-02],
        [-4.8749e-02, -4.1517e-01,  6.2532e-02,  5.8355e-01, -6.0003e-02,
          1.1662e-03, -2.4047e-02, -8.1129e-01, -4.1207e-02, -1.1157e-01],
        [-8.1197e-01, -3.0749e-01,  2.0993e-01,  4.0614e-01, -2.5502e-02,
          1.2981e-01, -7.2547e-02, -7.8023e-01, -3.8926e-01, -3.2443e-02],
        [-1.3424e-01, -1.2341e-02, -3.3536e-01,  3.1142e-01,  3.8534e-01,
          1.1562e-01,  3.4301e-02, -8.3104e-01, -3.9258e-02, -2.3927e-01],
        [-4.7431e-01, -1.2098e-02, -1.1135e-01,  6.4682e-01, -2.8632e-02,
          2.6898e-01, -3.3253e-0

In [57]:
critreion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.01)

In [58]:
def accuracy(y_hat,y):
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1 :
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y   
    return float(cmp.type(y.dtype).sum())

In [60]:
Epoch = 3 

for step in range(Epoch):
    for X,y in Data_T:
        #X= X.reshape(X.shape[0],-1)
        pred = model(X)
        loss = critreion(pred , y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc = accuracy(model(X), y )
        print(f" corrspnding to loss eval {loss.item()}, acc : {acc}")

torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 2.5027236938476562, acc : 17.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 2.0541257858276367, acc : 18.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 1.8441741466522217, acc : 20.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 2.002833604812622, acc : 23.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 1.6429049968719482, acc : 21.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 1.6801903247833252, acc : 24.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 1.660139799118042

torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 1.10548734664917, acc : 26.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.6986380815505981, acc : 26.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.7645474672317505, acc : 27.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.8897403478622437, acc : 24.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.4869539439678192, acc : 30.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.7494605779647827, acc : 26.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.6210418939590454

torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.6857497692108154, acc : 26.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.5443006157875061, acc : 29.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.8221874833106995, acc : 22.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.8027496337890625, acc : 23.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.6384320259094238, acc : 26.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.7614162564277649, acc : 26.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.43488866090774

torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.8339641094207764, acc : 25.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.6976544260978699, acc : 27.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.6101732850074768, acc : 26.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.3614443838596344, acc : 28.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.6560872197151184, acc : 27.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.5213890075683594, acc : 27.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 1.0768142938613892, acc : 26.0
torch.Size([32, 3072])
torch.

torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.577710747718811, acc : 26.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.5918148159980774, acc : 26.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.47477585077285767, acc : 31.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.42547979950904846, acc : 30.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.9433525204658508, acc : 27.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.5270266532897949, acc : 29.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.3363944292068

...

torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.6381508111953735, acc : 26.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.5667659044265747, acc : 27.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.3357267677783966, acc : 29.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.4995853900909424, acc : 27.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.335021048784256, acc : 28.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.46044090390205383, acc : 28.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.53098976612091

torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.7749781012535095, acc : 24.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.4785425066947937, acc : 28.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.36984220147132874, acc : 30.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.4552003741264343, acc : 29.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.356197714805603, acc : 28.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.41863685846328735, acc : 30.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.3747645616531372, acc : 28.0
torch.S

torch.Size([32, 10])
 corrspnding to loss eval 0.37923604249954224, acc : 29.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.28072988986968994, acc : 29.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.6214112043380737, acc : 29.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.2814059555530548, acc : 28.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.35099488496780396, acc : 29.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.18188273906707764, acc : 31.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.38528937101364136, acc : 29.0
torch.Size([32, 3072])
torch.Size([32, 10])
tor

torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.3774450719356537, acc : 27.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.43782365322113037, acc : 29.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.48125526309013367, acc : 27.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.5583686232566833, acc : 28.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.14637240767478943, acc : 32.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.3845840096473694, acc : 29.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.26127743721

torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.547803521156311, acc : 27.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.24654729664325714, acc : 30.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.17222051322460175, acc : 30.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.5997052788734436, acc : 29.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.45897984504699707, acc : 28.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.2659170627593994, acc : 30.0
torch.Size([32, 3072])
torch.Size([32, 10])
torch.Size([32, 3072])
torch.Size([32, 10])
 corrspnding to loss eval 0.523021638393

In [104]:
test_net = nn.Sequential(nn.Linear(2,4),nn.Linear(4,1))
X = torch.rand(3,2)
model_et = test_net(X)
model_et.max(dim=0)

torch.return_types.max(
values=tensor([0.0989], grad_fn=<MaxBackward0>),
indices=tensor([1]))

In [67]:
test_net.parameters

<bound method Module.parameters of Sequential(
  (0): Linear(in_features=2, out_features=4, bias=True)
  (1): Linear(in_features=4, out_features=1, bias=True)
)>

In [65]:
print(test_net[0].state_dict())

OrderedDict([('weight', tensor([[-0.3223, -0.3624],
        [-0.6307, -0.6587],
        [-0.5803, -0.2122],
        [-0.1346,  0.1910]])), ('bias', tensor([-0.3720,  0.5121, -0.3538,  0.5798]))])


In [9]:
######### this building sequential layers blocks of NET 
def coonv1():
    L_n1 = nn.Sequential(nn.Linear(3,4),nn.ReLU())
    return L_n1 

def Neasted_net():
    net = nn.Sequential()
    for i in range(2):
        net.add_module(f"block {i}",coonv1())
    return net     

In [10]:
Model = nn.Sequential(Neasted_net(),nn.Linear(4,1))

In [11]:
Model.parameters

<bound method Module.parameters of Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=3, out_features=4, bias=True)
      (1): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=3, out_features=4, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)>

In [12]:
Model.state_dict()

OrderedDict([('0.block 0.0.weight',
              tensor([[ 0.2645, -0.3581, -0.3310],
                      [-0.3550,  0.0031,  0.1263],
                      [-0.2866,  0.0893, -0.3221],
                      [-0.0870,  0.0590,  0.1854]])),
             ('0.block 0.0.bias',
              tensor([ 0.4216, -0.5089,  0.0543, -0.2628])),
             ('0.block 1.0.weight',
              tensor([[-0.5361, -0.4292, -0.4704],
                      [ 0.0156,  0.1526, -0.4022],
                      [ 0.3608, -0.2771,  0.3031],
                      [-0.5385,  0.4132,  0.1657]])),
             ('0.block 1.0.bias',
              tensor([-0.0402, -0.1179, -0.2704,  0.1794])),
             ('1.weight', tensor([[-0.0961,  0.2297,  0.1764, -0.0323]])),
             ('1.bias', tensor([0.0953]))])

In [13]:
#### intialization the wieghts of net 
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight,mean = 0 , std=0.1)
        nn.init.zeros_(m.bias)
    return init_weights     


In [14]:
Model.apply(init_weights)
Model[0][0][0].weight.data[0],Model[0][1][0].bias.data[0]

(tensor([ 0.0660, -0.0577, -0.0733]), tensor(0.))

##### Regulization and costume loss to traning regression model 

In [161]:
"""Create generated dataset to training"""
def data_generator(Samples, Weights, bias):
    X = torch.normal(mean = 0.1,std=0.01,size=(Samples,len(Weights)))
    Y = torch.matmul(X,Weights) + bias
    Y += torch.normal(0.1,0.01,size= Y.shape)
    return X , Y.reshape(-1,1)

In [162]:
n_train,n_test, input_ , batch = 20,200,100,5

In [163]:
w , b = torch.ones((input_, 1)) * 0.1 , 0.5

In [164]:
train = data_generator(n_train , w , b)
test = data_generator(n_test , w , b )

In [165]:
f"traninf size : {train[0].shape} testing size {train[1].shape} "

'traninf size : torch.Size([20, 100]) testing size torch.Size([20, 1]) '

In [166]:
def dataloader(train, batch, is_train=True):
    data = TensorDataset(*train)
    return DataLoader(dataset=data , batch_size=batch, shuffle=is_train)

In [167]:
training_set= dataloader(train, batch )
testing_set = dataloader(test, batch , is_train=False)

In [168]:
def init_param():
    w = torch.normal(1,0.1 ,size = (input_ , 1),requires_grad=True)
    print(w.grad)
    b = torch.zeros(1,requires_grad=True)
    return [w ,b ]

In [169]:
def forward(X , w ,b) :
    return torch.matmul(X, w ) + b 

In [170]:
def costume_Loss(pred_y , y ):
    return (pred_y - y.reshape(y.shape)) ** 2 /2 

In [172]:
"""intialzie the params """
w ,b = init_param()


None


In [173]:
def SGD(params, lr , batch):
    with torch.no_grad():
        for param in params:
            print(param.grad)
            param -= lr * param.grad / batch
            param.grad.zero_()

In [174]:
def Eval(regression ,data_iter,  loss ):
    Loss=0
    for X , Y in data_iter :
        pred_Y = regression(X)
        Y = Y.reshape(pred_Y.shape)
        Loss += loss(pred_Y, Y).sum()
    return Loss 

In [175]:
"""regulizer"""
def l2_penalty(w):
    return torch.sum(w.pow(2)) / 2

In [160]:
epochs , lr = 100, 0.01
alpha = 0.15
regression = lambda X: forward(X , w,b)
train_loss = np.zeros(epochs)
test_loss  = np.zeros(epochs)

"""itertion loop training"""

for epoch in range(epochs):
    for X , Y in training_set:
        pred_Y = regression(X)
        loss = costume_Loss(pred_Y, Y ) + alpha + l2_penalty(w)
        SGD([w,b],0.001, 5)
    with torch.no_grad():
        train_loss[epoch]=Eval(regression,training_set,costume_Loss)
        test_loss[epoch]=Eval(regression,testing_set,costume_Loss)

plt.rcParams.update({'font.size': 14})
plt.plot(np.arange(epochs), train_loss, label='train')
plt.plot(np.arange(epochs), test_loss, label='test')
plt.legend()
plt.yscale('log')

        

None


TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'